In [2]:
# python: base (3.11.4)

import numpy as np
import json
import pandas as pd
import time
import random

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback

from sklearn.metrics.pairwise import cosine_similarity

from tensorflow.keras.models import Model
import pandas as pd

from src.learner import *
from utilities import *

# data
kidwords = pd.read_csv('data/kidwords/kidwords.csv', header=None)[0].tolist()
kidwords_frequencies_from_tasa = pd.read_csv('data/kidword_frequencies_from_tasa.csv') # some words from kidwords aren't represented so we have to impute 0


top_500 = pd.read_csv('data/top_500.csv')
train_word_indices = np.array([i for i, e in enumerate(kidwords) if e in top_500['word'].values])


XX = np.genfromtxt('data/kidwords/orth.csv', delimiter=",")
YY = np.genfromtxt('data/kidwords/phon.csv', delimiter=",")

non_zero_a = np.any(XX != 0, axis=0)
X = XX[:, non_zero_a]

non_zero_b = np.any(YY != 0, axis=0)
Y = YY[:, non_zero_b]

# configs
with open('data/config.json', "r") as f:
    cfg = json.load(f)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU'))) # may not work depending on kernel

Num GPUs Available:  0


In [3]:
model = learner(X, Y, cfg['seed'], cfg['hidden_units'], optimizer='adam')
model.fit(X[train_word_indices], Y[train_word_indices], epochs=cfg['epochs'], batch_size=cfg['batch_size'], verbose=True)

Epoch 1/100
50/50 [==============================] - 1s 6ms/step - loss: 0.6774 - binary_accuracy: 0.5779 - mse: 0.2422
Epoch 2/100
50/50 [==============================] - 0s 6ms/step - loss: 0.5495 - binary_accuracy: 0.8333 - mse: 0.1808
Epoch 3/100
50/50 [==============================] - 0s 8ms/step - loss: 0.3837 - binary_accuracy: 0.8738 - mse: 0.1125
Epoch 4/100
50/50 [==============================] - 0s 7ms/step - loss: 0.3315 - binary_accuracy: 0.8714 - mse: 0.0985
Epoch 5/100
50/50 [==============================] - 0s 6ms/step - loss: 0.3144 - binary_accuracy: 0.8722 - mse: 0.0945
Epoch 6/100
50/50 [==============================] - 0s 9ms/step - loss: 0.2957 - binary_accuracy: 0.8757 - mse: 0.0890
Epoch 7/100
50/50 [==============================] - 0s 5ms/step - loss: 0.2904 - binary_accuracy: 0.8756 - mse: 0.0879
Epoch 8/100
50/50 [==============================] - 0s 5ms/step - loss: 0.2811 - binary_accuracy: 0.8798 - mse: 0.0850
Epoch 9/100
50/50 [=====================

In [4]:
# Create a new model that outputs the hidden layer activations
hidden_layer_model = Model(inputs=model.input, outputs=model.layers[0].output)

hidden_activations = hidden_layer_model.predict(X)

# Convert the activations to a DataFrame
df = pd.DataFrame(hidden_activations)

df.to_csv('outputs/pilot_1_hidden_unit_activations.csv')


In [8]:
np.min(similarities)

-0.39985177

In [10]:
# Compute the similarity matrix
similarities = cosine_similarity(hidden_activations)

np.savetxt('outputs/pilot_1_hidden_unit_activation_similarities.csv', similarities, delimiter = ",", fmt = '%.8f')

ValueError: unsupported format character ',' (0x2c) at index 3